# Tutorial: **δHBV 1.1p** (GEFS)

---

This notebook demonstrates training and forward simulation with the δHBV 1.1p model developed by [Yalan Song et al. (2025)](https://doi.org/10.22541/essoar.172304428.82707157/v2). A pre-trained model is provided for those who only wish to run the model forward.

For explanation of model structure, methodologies, data, and performance metrics, please refer to Song's publication [below](#publication). If you find this code is useful in your own work, please include the aforementioned citation.

**Note**: If you are new to the dMG framework, we suggest first looking at our [δHBV 1.0 tutorial](./../hydrology/example_dhbv.ipynb).

<br>

### Before Running:
- **Environment**: See [setup.md](./../../docs/setup.md) for ENV setup. dMG must be installed with dependencies + hydrodl2 to run this notebook.

- **Model**: Download pretrained δHBV 1.1p model weights from [AWS](https://mhpi-spatial.s3.us-east-2.amazonaws.com/mhpi-release/models/dhbv_1_1p_trained.zip). Then update the model config:

    - In [`./generic_deltamodel/example/conf/config_dhbv_1_1p.yaml`](./../conf/config_dhbv_1_1p.yaml), update *trained_model* with your path to the parent directory containing both trained model weights `dhbv_1_1p_ep50.pt` (or *Ep100*) **and** normalization file `normalization_statistics.json`.
    - **Note**: make sure this path includes the last closing forward slash: e.g., `./your/path/to/model/`.

- **Data**: Download the CAMELS data extraction from [AWS](https://mhpi-spatial.s3.us-east-2.amazonaws.com/mhpi-release/camels/camels_data.zip). Then, updated the data configs:

    - In [`./generic_deltamodel/example/conf/observations/camels_531.yaml`](./../conf/observations/camels_531.yaml) and [`camels_671.yaml`](./../conf/observations/camels_671.yaml), update...
        1. *data_path* with `camels_dataset` path,
        2. *gage_info* with `gage_ids.npy` path,
        3. *subset_path* with `531_subset.txt` path (camels_531 only).

    - The full 671-basin or 531-basin CAMELS datasets can be selected by setting `observations: camels_671` or `camels_531` in the model config, respectively.

- **Hardware**: The NNs used in this model require CUDA support only available with Nvidia GPUs. For those without access, T4 GPUs can be used when running this notebook with dMG on [Google Colab](https://colab.research.google.com/).

<br>

### Publication:

*Yalan Song, Kamlesh Sawadekar, Jonathan M Frame, Ming Pan, Martyn Clark, Wouter J M Knoben, Andrew W Wood, Trupesh Patel, Chaopeng Shen. "Physics-informed, Differentiable Hydrologic  Models for Capturing Unseen Extreme Events." ESS Open Archive (2025). https://doi.org/10.22541/essoar.172304428.82707157/v2.*

<br>

### Issues:
For questions, concerns, bugs, etc., please reach out by posting an [issue](https://github.com/mhpi/generic_deltamodel/issues).

---


<br>

## 1. Forward δHBV 1.1p

After completing [these](#before-running) steps, forward δHBV 1.1p with the code block below.

Note:
- The settings defined in the config `./generic_deltamodel/example/conf/config_dhbv_1_1p.yaml` are set to replicate benchmark performance on 531 CAMELS basins.
- The first year (`warm_up` in the config, default is 365 days) of the inference period is used for initializing HBV's internal states (water storages) and is, therefore, excluded from the model's prediction output.

### 1.1 Demonstration

In [ ]:
import sys

sys.path.append('../../')
from example import load_config
from src.dmg import ModelHandler
from src.dmg.core.utils import import_data_loader, print_config, set_randomseed

from ._gefs_utils import print_dataset_info

# ------------------------------------------#
# Define model settings here.
CONFIG_PATH = '../example/conf/config_dhbv_1_1p.yaml'
# ------------------------------------------#

# 1. Load configuration dictionary of model parameters and options.
config = load_config(CONFIG_PATH)
config['mode'] = 'simulation'
print_config(config)
print("Config Keys:", config.keys())

# Set random seed for reproducibility.
set_randomseed(config['random_seed'])

# 2. Initialize the differentiable HBV 1.1p model (LSTM + HBV 1.1p).
model = ModelHandler(config, verbose=True)  # model key is ['Hbv_1_1p']
nn_model = model.model_dict["Hbv_1_1p"].nn_model
print("NN model class:", type(nn_model).__name__)
print("Model Keys:", model.model_dict.keys())
print("Model Keys:", model.model_dict['Hbv_1_1p'])

# 3. Load and initialize a dataset dictionary of NN and HBV model inputs.
data_loader_cls = import_data_loader(config['data_loader'])
data_loader = data_loader_cls(config, test_split=False, overwrite=False)
print_dataset_info(data_loader.dataset)

### 1.2 Define Core GEFS Utilities

In [ ]:
import os

import numpy as np
import pandas as pd
import torch

from ._gefs_utils import GEFSdataErrorCheck, to_time_first


def compute_bias_correction_from_dataset(
    daymet_tensor,
    gefs_df,
    timesteps,
    basin_idx,
    window=15,
    method="scalar",
):
    """
    Compute bias correction factors (scalar or CDF-based) for GEFS forcings
    using Daymet climatology in a moving DOY window.

    Parameters
    ----------
    daymet_tensor : torch.Tensor [time, basin, 3]
        Historical Daymet forcings from the dataset.
    gefs_df : pd.DataFrame
        GEFS historical forcings with columns: date, prcp, tmean, pet.
    timesteps : np.ndarray
        Daily time axis corresponding to Daymet.
    basin_idx : int
        Basin index to select from Daymet tensor.
    window : int
        ±days around each DOY to compute moving-window stats.
    method : str, {"scalar", "cdf"}
        Bias correction method:
        - "scalar": mean ratio (current method)
        - "cdf": quantile mapping / CDF matching

    Returns
    -------
    dict
        A dictionary of corrections per variable.
        For "scalar" → corrections[var][doy] = scalar value
        For "cdf" → corrections[var][doy] = (p_grid, mapped_values)
    """
    # Convert Daymet tensor → DataFrame
    df_daymet = pd.DataFrame(
        {
            "date": pd.to_datetime(timesteps),
            "prcp": daymet_tensor[:, basin_idx, 0].detach().cpu().numpy(),
            "tmean": daymet_tensor[:, basin_idx, 1].detach().cpu().numpy(),
            "pet": daymet_tensor[:, basin_idx, 2].detach().cpu().numpy(),
        },
    )
    df_daymet["doy"] = df_daymet["date"].dt.dayofyear

    gefs_df = gefs_df.copy()
    gefs_df["doy"] = gefs_df["date"].dt.dayofyear

    corrections = {v: {} for v in ["prcp", "tmean", "pet"]}

    for var in ["prcp", "tmean", "pet"]:
        for doy in range(1, 366):
            mask_d = (df_daymet["doy"] - doy).abs() <= window
            mask_g = (gefs_df["doy"] - doy).abs() <= window

            vals_d = df_daymet.loc[mask_d, var].dropna()
            vals_g = gefs_df.loc[mask_g, var].dropna()

            if len(vals_d) < 10 or len(vals_g) < 10:
                continue

            if method == "scalar":
                mean_d = vals_d.mean()
                mean_g = vals_g.mean()
                corrections[var][doy] = mean_d / mean_g if mean_g > 1e-6 else 1.0

            elif method == "cdf":
                # Create percentile grid (0–100)
                p_grid = np.linspace(0, 100, 101)
                q_daymet = np.percentile(vals_d, p_grid)
                q_gefs = np.percentile(vals_g, p_grid)
                corrections[var][doy] = (q_gefs, q_daymet)  # (x_in, x_out)
            else:
                raise ValueError(f"Unknown correction method '{method}'")

    return corrections


def pre_processing(df, corrections, method="scalar"):
    """Apply bias correction (scalar or CDF) to GEFS forecast DataFrame."""
    df = df.copy()
    df["doy"] = df["date"].dt.dayofyear

    for var in ["prcp", "tmean", "pet"]:
        if method == "scalar":
            df[var] *= df["doy"].map(lambda d, var=var: corrections[var].get(d, 1.0))
        elif method == "cdf":
            corrected_vals = []
            for _, row in df.iterrows():
                doy = row["doy"]
                val = row[var]
                if doy not in corrections[var]:
                    corrected_vals.append(val)
                    continue
                q_in, q_out = corrections[var][doy]
                # interpolate within empirical CDFs
                corrected_vals.append(np.interp(val, q_in, q_out))
            df[var] = corrected_vals

        else:
            raise ValueError(f"Unknown correction method '{method}'")

    return df


def post_processing(sim_pre_GEFS, ens_preds):  # post processing
    """
    Apply a constant offset correction so that the first forecasted streamflow
    connects smoothly with the last simulated (Daymet) streamflow.

    Parameters
    ----------
    sim_pre_GEFS : np.ndarray
        1D array of simulated streamflow before the forecast period (from HBV using Daymet).
    ens_preds : np.ndarray
        2D array of ensemble forecasts with shape (N_ENSEMBLES, FORECAST_LENGTH).

    Returns
    -------
    np.ndarray
        Offset-corrected ensemble forecasts of the same shape.
    """
    Q_sim_end = sim_pre_GEFS[-1]  # last simulated streamflow value
    ens_preds_corrected = []

    for ens in ens_preds:
        offset = Q_sim_end - ens[0]  # compute offset
        ens_corrected = np.maximum(ens + offset, 0.0)  # apply and ensure non-negative
        ens_preds_corrected.append(ens_corrected)

    return np.array(ens_preds_corrected)


def run_warm_forecasts(
    hbv,
    pars_last,
    rtwts_hist,
    warm_states,
    gage_id,
    start_date,
    horizon,
    varF,
    N_ENSEMBLES,
    GEFS_DIR,
    timesteps,
    WINDOW,
    CORRECTION,
    device,
):
    """Run warm-started HBV forecasts for all GEFS ensembles."""
    ens_preds = []
    # will store per-basin correction once

    for ens_id in range(N_ENSEMBLES):
        # ----- Load GEFS forecast for this ensemble -----
        f_path = os.path.join(GEFS_DIR, f"ens0{ens_id + 1}", f"{gage_id:08d}.txt")
        if not os.path.exists(f_path):
            raise ValueError(f"Missing GEFS file: {f_path}")

        df = pd.read_csv(f_path, sep=r"\s+", header=0)
        df = df.rename(
            columns={
                "Year": "year",
                "Mnth": "month",
                "Day": "day",
                "prcp(mm/day)": "prcp",
                "tmean(C)": "tmean",
                "pet(mm/day)": "pet",
            },
        )
        df["date"] = pd.to_datetime(df[["year", "month", "day"]])

        # ----- Extract forecast block -----
        idx_list = df.index[df["date"] == start_date].to_list()
        if not idx_list:
            raise ValueError(f"Start date {start_date} not found in {f_path}")
        fc_block = df.iloc[idx_list[0] : idx_list[0] + horizon]
        GEFSdataErrorCheck(idx_list, fc_block, horizon, start_date)

        # ----- Compute bias correction once (for the first ensemble) -----
        if not CORRECTION:
            bias_corrections = compute_bias_correction_from_dataset(
                dataset["x_phy"],
                df,
                timesteps,
                basin_idx,
                window=WINDOW,
                method=CORRECTION,  # or "scalar"
            )
            print(f"Bias correction computed for basin {gage_id}")
            # ----- Apply bias correction -----
            fc_block = pre_processing(fc_block, bias_corrections, method=CORRECTION)

        # ----- Convert forcings to tensor -----
        raw_np = fc_block[varF].to_numpy().astype(np.float32)
        forc_raw = torch.tensor(
            raw_np[np.newaxis, :, :],
            dtype=torch.float32,
            device=device,
        )
        forc_raw[torch.isnan(forc_raw)] = 0.0

        # ----- Run HBV forecast with warm states -----
        with torch.no_grad():
            Qs_fc = hbv(
                x=to_time_first(forc_raw),
                parameters=pars_last,
                staind=staind,
                tdlst=tdRep,
                mu=nmul,
                muwts=None,
                rtwts=rtwts_hist,
                bufftime=0,
                outstate=False,
                instate=True,
                init_states=warm_states,
                routOpt=routing,
                dydrop=dydrop,
            )

        # ----- Sanity check and store -----
        ens_fc = Qs_fc[:, 0, 0].detach().cpu().numpy()
        if np.isnan(ens_fc).all():
            raise ValueError(
                f"ens_fc are all NaNs for basin {gage_id}, ensemble {ens_id}!",
            )

        ens_preds.append(ens_fc)

    return np.array(ens_preds)  # shape: (N_ENSEMBLES, horizon)

### 1.3 GEFS Simulation

In [ ]:
import json
import os

import numpy as np
import pandas as pd
import torch
from hydrodl2.models.hbv import HBV1_1p as h1pp

from ._gefs_utils import (
    get_parameters_from_model,
    plot_ensemble_hydrograph,
    selectbasins,
    startid_endid,
)

# ---------------- Simulation settings ---------------- #
START_DATE = pd.to_datetime("2014-09-16")
FORECAST = 15
N_ENSEMBLES = 5
N_BASINS = 5  # randomly pick basins
SEED = 1234  # config["random_seed"]
RANDOM = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
GEFS_DIR = "/u/st/dr/awwood/aw-ciroh-proj/projects/dl_da/daymet-gefs-camels-gII"
GAGE_NAME_PATH = "/beegfs/projects/aw-ciroh/common/datasets/camels/CAMELS_US/camels_attributes_v2.0/camels_name.txt"
dataset = data_loader.dataset
WINDOW = 30  # pre-processing on correcting window (in days)
CORRECTION = 'scalar'  #  # or 'cdf'
# ----------------------------------------------------- #


# HBV options
staind = -1
tdRep = [1, 3, 13]
VAR_F = config['delta_model']['phy_model']['forcings']
BUFFTIME = config['delta_model']['phy_model']['warm_up']
routing = config['delta_model']['phy_model']['routing']
dydrop = config['delta_model']['phy_model']['dy_drop']
nmul = config["delta_model"]["phy_model"]["nmul"]

# Plot settings
history_len = 120


# ---------------- Obtain basin IDs ---------------- #
gage_ids = np.load(config["observations"]["gage_info"], allow_pickle=True)
subset_file = config["observations"]["subset_path"]
with open(subset_file) as f:
    content = f.read().strip()
gage_ids_subset = (
    json.loads(content)
    if content.startswith("[")
    else np.loadtxt(subset_file, dtype=int).tolist()
)

if config["observations"]["name"] == "camels_671":
    basin_pool = list(gage_ids)
elif config["observations"]["name"] == "camels_531":
    basin_pool = list(gage_ids_subset)


# ---------------- Select 5 random basins ---------------- #
selected_basins = selectbasins(RANDOM, SEED, basin_pool, N_BASINS)
sidx, eidx, timesteps = startid_endid(START_DATE, FORECAST, config)
hbv = h1pp.HBV().to(device)


# ---------------- Loop over selected basins ---------------- #
for GAGE_ID in selected_basins:
    print(f"\n=== Running forecast for basin {GAGE_ID} ===")
    basin_idx = basin_pool.index(GAGE_ID)

    # ---------------- Get inputs of all forcings 38 from trained NN ---------------- #
    hist_dict = {
        "xc_nn_norm": dataset["xc_nn_norm"][
            :sidx,
            basin_idx : basin_idx + 1,
            :,
        ].clone(),
    }  # [715, 1, 38] 2years - 15 days = 715 days
    print(f"hist_dict['xc_nn_norm'] shape: {hist_dict['xc_nn_norm'].shape}")

    # ---------------- Get parameters from trained NN ---------------- #
    pars_hist, rtwts_hist = get_parameters_from_model(
        model.model_dict["Hbv_1_1p"],
        hist_dict,
        n_par=14,
        mu=nmul,
        device=device,
    )  # pars: 715, 1, 14, 16    rts: 1, 2

    pars_first = pars_hist[:1].repeat(sidx, 1, 1, 1)
    pars_last = pars_hist[-1:].repeat(FORECAST, 1, 1, 1)

    # --- Warm-up HBV with RAW forcings (Daymet) ---
    with torch.no_grad():
        forc_hist = torch.nan_to_num(
            dataset["x_phy"][:sidx, basin_idx : basin_idx + 1, :3],
            nan=0.0,
        )
        Qs_hist, sp, mw, sm, suz, slz = hbv(
            x=forc_hist,
            parameters=pars_first,
            staind=staind,
            tdlst=tdRep,
            mu=nmul,
            muwts=None,
            rtwts=rtwts_hist,
            bufftime=BUFFTIME,
            outstate=True,
            instate=False,
            routOpt=routing,
            dydrop=dydrop,
        )
    warm_states = (sp, mw, sm, suz, slz)  # checknans(warm_states)

    # ---------------- Run ensembles ---------------- #
    ens_preds = run_warm_forecasts(
        hbv,
        pars_last,
        rtwts_hist,
        warm_states,
        GAGE_ID,
        START_DATE,
        FORECAST,
        VAR_F,
        N_ENSEMBLES,
        GEFS_DIR,
        timesteps,
        WINDOW,
        CORRECTION,
        device,
    )

    # ---------------- Plotting ---------------- #
    obs_full_window = (
        dataset["target"][sidx - history_len : eidx, basin_idx, 0]
        .detach()
        .cpu()
        .numpy()
    )

    Qs_full = Qs_hist[:, 0, 0].detach().cpu().numpy()
    sim_pre_GEFS = Qs_full[-(history_len + 1) :]
    ens_preds = post_processing(sim_pre_GEFS, ens_preds)  # post-processing

    save_path = os.path.join(
        "figs",
        f"GEFS_{GAGE_ID}",
        f"GEFS_{GAGE_ID}_{CORRECTION}_w{WINDOW}.png",
    )
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    plot_ensemble_hydrograph(
        gage_path=GAGE_NAME_PATH,
        gage_id=GAGE_ID,
        start_date=START_DATE,
        obs=obs_full_window,
        sim=sim_pre_GEFS,
        ens_preds=ens_preds,
        history_len=history_len,
        save_path=save_path,
    )
    print(f"Saved plot for basin {GAGE_ID} → {save_path}")

In [ ]:
# If you want to simulate on the whole range using daymet (nothing to do with GEFS)
output = model(
    data_loader.dataset,
    eval=True,
)

print(
    "-------------\n",
)  # only after warmup, only output['Hbv_1_1p']['streamflow'] is meaningful
print(
    f"Streamflow predictions for {output['Hbv_1_1p']['streamflow'].shape[0]} days (after warmup) and "
    f"{output['Hbv_1_1p']['streamflow'].shape[1]} basins ~ \nShowing the first 5 days for "
    f"first basin: \n {output['Hbv_1_1p']['streamflow'][:5, :1].cpu().detach().numpy().squeeze()}",
)

### 1.4 Visualizing Model Predictions Only

After running model inference we can, e.g., view the hydrograph for one of the basins to see we are getting expected outputs.

We can do this with our target variable, streamflow, for instance (though, there are many other states and fluxes we can view -- see cell output below).

In [ ]:
import numpy as np
import pandas as pd
from src.dmg.core.post import plot_hydrograph
from src.dmg.core.utils import Dates

from ._gefs_utils import obtain_gage_name

# ------------------------------------------#
# Choose a basin by USGS gage ID to plot.
GAGE_ID = 1022500

# Resample to 3-day prediction. Options: 'D', 'W', 'M', 'Y'.
RESAMPLE = '3D'
# ------------------------------------------#

gage_ids_531 = np.load('./gage_ids_531.npy', allow_pickle=True)


# 1. Get the streamflow predictions and daily timesteps of the prediction window.
pred = output['Hbv_1_1p']['streamflow']
timesteps = Dates(
    config['simulation'],
    config['delta_model']['rho'],
).batch_daily_time_range
timesteps = timesteps[
    config['delta_model']['phy_model']['warm_up'] :
]  # remove warm-up period
if config['observations']['name'] == 'camels_671':
    basin_idx = list(gage_ids).index(GAGE_ID) if GAGE_ID in gage_ids else None
elif config['observations']['name'] == 'camels_531':
    basin_idx = (
        list(gage_ids_531).index(GAGE_ID) if GAGE_ID in gage_ids_subset else None
    )
else:
    raise ValueError(f"Unsupported dataset: {config['observations']}")

# 2. Get the data for the chosen basin and plot.
streamflow_pred_basin = pred[:, basin_idx].squeeze()
gage_name = obtain_gage_name(GAGE_NAME_PATH, GAGE_ID)
plot_hydrograph(
    timesteps,
    streamflow_pred_basin,
    resample=RESAMPLE,
    title=f"Hydrograph for Gage ID {GAGE_ID} ({gage_name})",
    ylabel='Streamflow (mm/day)',
)

<br>

## 2. Train δHBV 1.1p 

- Train δHBV 1.1p with the code block below.
- If you have already trained data, skip this.
- If you have not trained data nor download pretrained data, you won't be able to accomplish Step 1!

**Note**
- The settings defined in the config `./generic_deltamodel/example/conf/config_dhbv_1_1p.yaml` are set to replicate benchmark performance.
- For model training, set `mode: train` in the config, or modify after config dict has been created (see below).
- An `./example/generic_deltamodel/output/` directory will be generated to store experiment and model files. This location can be adjusted by changing the *save_path* key in your config. 
- Default settings with 50 epochs, batch size of 100, and training window from 1 October 1999 to 30 September 2008 should use ~2.8GB of vram. Expect training times of ~9 hours with an Nvidia RTX 3090 Ti.

In [ ]:
import sys

sys.path.append('../../')

from example import load_config

from dmg import ModelHandler
from dmg.core.utils import (
    import_data_loader,
    import_trainer,
    print_config,
    set_randomseed,
)

# ------------------------------------------#
# Define model settings here.
CONFIG_PATH = '../example/conf/config_dhbv_1_1p.yaml'
# ------------------------------------------#

# 0. zhennan added --- tiny tee: mirror all prints/progress to train.log ---
import sys

from tqdm import tqdm

# tqdm writes to stderr by default; redirect to file also
logfile = open("train_progress.log", "a", buffering=1)


class TqdmLogger:
    """Logger class to redirect tqdm output to multiple files."""

    def __init__(self, *files):
        self.files = files

    def write(self, x):
        """Write string x to all files."""
        for f in self.files:
            f.write(x)
            f.flush()

    def flush(self):
        """Flush all files."""
        for f in self.files:
            f.flush()


sys.stderr = TqdmLogger(sys.stderr, logfile)  # capture tqdm progress bar


# 1. Load configuration dictionary of model parameters and options.
config = load_config(CONFIG_PATH)
config['mode'] = 'train'
print_config(config)

# Set random seed for reproducibility.
set_randomseed(config['random_seed'])

# 2. Initialize the differentiable HBV 1.1p model (LSTM + HBV 1.1p) with model handler.
model = ModelHandler(config, verbose=True)

# 3. Load and initialize a dataset dictionary of NN and HBV model inputs.
data_loader_cls = import_data_loader(config['data_loader'])
data_loader = data_loader_cls(config, test_split=True, overwrite=False)


# 4. Initialize trainer to handle model training.
trainer_cls = import_trainer(config['trainer'])
trainer = trainer_cls(
    config,
    model,
    train_dataset=data_loader.train_dataset,
)

# 5. Start model training.
trainer.train()
print(f"Training complete. Model saved to {config['model_path']}")

## 3. Evaluate Model Performance

After completing the training in [Section 1](#1-train-δhbv-11p), or with the trained model provided, test δHBV 1.1p below on the evaluation data.

**Note**
- For model evaluation, set `mode: test` in the config, or modify after config dict has been created (see below).
- When evaluating provided models, confirm that `test.test_epoch` in the config corresponds the training epochs completed for the model you want to test (e.g., 50 or 100).
- Default settings with 50 epochs, batch size of 25, and testing window from 1 October 1989 to 30 September 1999 should use ~2.7GB of VRAM. Expect evalutation times of ~5 minutes with an Nvidia RTX 3090 Ti.

### 3.1 Streamflow Simulation

In [ ]:
import sys

sys.path.append('../../')

from example import load_config

from dmg import ModelHandler
from dmg.core.utils import import_data_loader, import_trainer, print_config

# ------------------------------------------#
# Define model settings here.
CONFIG_PATH = '../example/conf/config_dhbv_1_1p.yaml'
# ------------------------------------------#


# 1. Load configuration dictionary of model parameters and options.
config = load_config(CONFIG_PATH)
config['mode'] = 'test'
print_config(config)

set_randomseed(config['random_seed'])

# 2. Initialize the differentiable HBV 1.1p model (LSTM + HBV 1.1p).
model = ModelHandler(config, verbose=True)

# 3. Load and initialize a dataset dictionary of NN and HBV model inputs.
data_loader_cls = import_data_loader(config['data_loader'])
data_loader = data_loader_cls(config, test_split=True, overwrite=False)

# 4. Initialize trainer to handle model evaluation.
trainer_cls = import_trainer(config['trainer'])
trainer = trainer_cls(
    config,
    model,
    eval_dataset=data_loader.eval_dataset,
    verbose=True,
)

# 5. Start testing the model.
print('Evaluating model...')
trainer.evaluate()
print(f"Metrics and predictions saved to {config['out_path']}")

### 3.2 Visualize Trained Model Performance

Once the model has been evaluated, a new directory (e.g., for a model trained for 50 epochs and tested from years 1989-1999), `test1989-1999_ep50/`, will be created in the same directory containing the model files. This path will be populated with...

1. All model outputs (fluxes, states), including the target variable, *streamflow* (`streamflow.npy`),

2. `streamflow_obs.npy`, streamflow observation data for comparison against model predictions,

2. `metrics.json`, containing evaluation metrics accross the test time range for every gage in the dataset,

3. `metrics_agg.json`, containing evaluation metric statistics across all sites (mean, median, standard deviation).

We can use these outputs to visualize δHBV 1.1p's performance with a 
1. Cumulative distribution function (CDF) plot, 

2. CONUS map of gage locations and metric (e.g., NSE) performance.

<br>

But first, let's first check the (basin-)aggregated metrics for NSE, KGE, bias, RMSE, and, for both high/low flow regimes, RMSE and absolute percent bias...

In [ ]:
import os

from dmg.core.data import load_json
from dmg.core.post import print_metrics

print(f"Evaluation output files saved to: {config['out_path']} \n")

# 1. Load the basin-aggregated evaluation results.
metrics_path = os.path.join(config['out_path'], 'metrics_agg.json')
metrics = load_json(metrics_path)
print(f"Available metrics: {metrics.keys()} \n")

# 2. Print the evaluation results.
metric_names = [
    # Choose metrics to show.
    'nse',
    'kge',
    'bias',
    'rmse',
    'rmse_low',
    'rmse_high',
    'flv_abs',
    'fhv_abs',
]
print_metrics(metrics, metric_names, mode='median', precision=3)

### 3.3 CDF Plot

The cumulative distribution function (CDF) plot tells us what percentage (CDF on the y-axis) of basins performed at least better than a given metric on the evaluation data.

An example is given below for NSE, but you can change to your preferred metric (see the output from the previous cell), but note some may require changing *xbounds* in `plot_cdf()`.

In [ ]:
from dmg.core.post import plot_cdf

# ------------------------------------------#
# Choose the metric to plot. (See available metrics printed above, or in the metrics_agg.json file).
METRIC = 'nse'
# ------------------------------------------#


# 1. Load the evaluation metrics.
metrics_path = os.path.join(config['out_path'], 'metrics.json')
metrics = load_json(metrics_path)

# 2. Plot the CDF for NSE.
plot_cdf(
    metrics=[metrics],
    metric_names=[METRIC],
    model_labels=['δHBV 1.1p'],
    title="CDF of NSE for δHBV 1.1p",
    xlabel=METRIC.capitalize(),
    figsize=(8, 6),
    xbounds=(0, 1),
    ybounds=(0, 1),
    show_arrow=True,
)

### 3.4 Spatial Plot

This plot shows the locations of each basin in the evaluation data, color-coded by performance on a metric. Here we give a plot for NSE, but as before, this can be changed to your preference. (See above; for metrics not valued between 0 and 1, you will need to set `dynamic_colorbar=True` in `geoplot_single_metric` to ensure proper coding.)

Note, you will need to add paths to the CAMELS shapefile, gage IDs, and 531-gage subset which can be found in the [CAMELS download](#before-running).

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd

from dmg.core.data import txt_to_array
from dmg.core.post import geoplot_single_metric

# ------------------------------------------#
# Choose the metric to plot. (See available metrics printed above, or in the metrics_agg.json file).
METRIC = 'nse'

# Set the paths to the gage id lists and shapefiles...
GAGE_ID_PATH = config['observations']['gage_info']  # ./gage_id.npy
GAGE_ID_531_PATH = config['observations']['subset_path']  # ./531sub_id.txt
SHAPEFILE_PATH = '/beegfs/scratch/zhennanshi/DM/generic_deltamodel/extra_files_zhennan/camels_loc/camels_671_loc.shp'  ## zhennan added to his directory

# ------------------------------------------#


# 1. Load gage ids + basin shapefile with geocoordinates (lat, long) for every gage.
gage_ids = np.load(GAGE_ID_PATH, allow_pickle=True)
gage_ids_subset = txt_to_array(GAGE_ID_531_PATH)
coords = gpd.read_file(SHAPEFILE_PATH)

# 2. Format geocoords for 531- and 671-basin CAMELS sets.
coords_531 = coords[coords['gage_id'].isin(list(gage_ids_subset))].copy()

coords['gage_id'] = pd.Categorical(
    coords['gage_id'],
    categories=list(gage_ids),
    ordered=True,
)
coords_531['gage_id'] = pd.Categorical(
    coords_531['gage_id'],
    categories=list(gage_ids_subset),
    ordered=True,
)

coords = coords.sort_values('gage_id')  # Sort to match order of metrics.
basin_coords_531 = coords_531.sort_values('gage_id')

# 3. Load the evaluation metrics.
metrics_path = os.path.join(config['out_path'], 'metrics.json')
metrics = load_json(metrics_path)

# 4. Add the evaluation metrics to the basin shapefile.
if config['observations']['name'] == 'camels_671':
    coords[METRIC] = metrics[METRIC]
    full_data = coords
elif config['observations']['name'] == 'camels_531':
    coords_531[METRIC] = metrics[METRIC]
    full_data = coords_531
else:
    raise ValueError(
        f"Observation data supported: 'camels_671' or 'camels_531'. Got: {config['observations']}",
    )

# 5. Plot the evaluation results spatially.
geoplot_single_metric(
    full_data,
    METRIC,
    rf"Spatial Map of {METRIC.upper()} for δHBV 1.1p on CAMELS "
    f"{config['observations']['name'].split('_')[-1]}",
    dynamic_colorbar=False,
)